<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [1]:
# Install spark

In [2]:
!pip install pyspark
!pip install findspark

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [ ]:
# Start session

In [5]:
sc = SparkContext()

spark = SparkSession.builder.appName("Ecommerce Search Term").getOrCreate()

23/12/24 07:34:48 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/24 07:34:50 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
# Download The search term dataset from the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

In [6]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

--2023-12-24 07:35:15--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233457 (228K) [text/csv]
Saving to: ‘searchterms.csv’

searchterms.csv     100%[===================>] 227.99K  --.-KB/s    in 0.004s  

2023-12-24 07:35:15 (50.9 MB/s) - ‘searchterms.csv’ saved [233457/233457]



In [7]:
# Load the csv into a spark dataframe

In [8]:
df = spark.read.csv("searchterms.csv", header=True, inferSchema=True)

In [9]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)

In [13]:
n_rows = df.count()
n_cols = len(df.columns)
print(f'Rows: {n_rows}')
print(f'Cols: {n_cols}')

Rows: 10000
Cols: 4


In [ ]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

In [14]:
df.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [12]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [19]:
col_name, col_dtype = df.select("searchterm").dtypes[0]
print(f'Column name: {col_name}')
print(f'Column data type: {col_dtype}')

Column name: searchterm
Column data type: string


In [20]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [21]:
df.createTempView("searchterms_data")

In [23]:
spark.sql("SELECT COUNT(*) FROM searchterms_data WHERE searchterm = 'gaming laptop'").show()

+--------+
|count(1)|
+--------+
|     499|
+--------+



In [24]:
df.filter(df.searchterm == 'gaming laptop').count()

499

In [25]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [34]:
spark.sql("""
SELECT searchterm, COUNT(*) AS frequency FROM searchterms_data
GROUP BY searchterm ORDER BY frequency DESC
LIMIT 5
""").show()

[Stage 37:====================================>                (138 + 17) / 200]

+-------------+---------+
|   searchterm|frequency|
+-------------+---------+
|mobile 6 inch|     2312|
|    mobile 5g|     2301|
|mobile latest|     1327|
|       laptop|      935|
|  tablet wifi|      896|
+-------------+---------+



In [37]:
df.groupBy("searchterm").agg({"searchterm": "count"}).withColumnRenamed("count(searchterm)", "frequency")\
    .sort("frequency", ascending=False).limit(5)\
    .show()

[Stage 43:=====================================>               (142 + 11) / 200]

+-------------+---------+
|   searchterm|frequency|
+-------------+---------+
|mobile 6 inch|     2312|
|    mobile 5g|     2301|
|mobile latest|     1327|
|       laptop|      935|
|  tablet wifi|      896|
+-------------+---------+



In [38]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

In [39]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
!tar -xvzf model.tar.gz

--2023-12-24 08:55:24--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1490 (1.5K) [application/x-tar]
Saving to: ‘model.tar.gz’

model.tar.gz        100%[===================>]   1.46K  --.-KB/s    in 0s      

2023-12-24 08:55:24 (10.6 MB/s) - ‘model.tar.gz’ saved [1490/1490]

sales_prediction.model/
sales_prediction.model/metadata/
sales_prediction.model/metadata/part-00000
sales_prediction.model/metadata/.part-00000.crc
sales_prediction.model/metadata/_SUCCESS
sales_prediction.model/metadata/._SUCCESS.crc
sales_prediction

In [40]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)

In [41]:
from pyspark.ml.regression import LinearRegressionModel

model = LinearRegressionModel.load('sales_prediction.model')

In [42]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

In [43]:
from pyspark.ml.feature import VectorAssembler

def predict(year):
    assembler = VectorAssembler(inputCols=["year"], outputCol="features")
    data = [[year, 0]]
    columns = ["year", "sales"]
    _ = spark.createDataFrame(data, columns)
    __ = assembler.transform(_).select('features', 'year')
    predictions = model.transform(__)
    predictions.select('prediction').show()

In [44]:
predict(2023)

+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+



23/12/24 09:00:03 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/12/24 09:00:03 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
